In [1]:
print("hello")

hello


In [ ]:
from langchain_pinecone import PineconeVectorStore
from langchain_core.prompts import PromptTemplate
from langchain_community.llms import CTransformers
from langchain_classic.chains import RetrievalQA
from langchain_community.document_loaders import PyMuPDFLoader, DirectoryLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings
from pinecone import Pinecone, ServerlessSpec

# from langchain import PromptTemplate
# from langchain.chains import RetrievalQA
# from langchain.embeddings import HuggingFaceEmbeddings
# from langchain_pinecone import PineconeVectorStore
# from pinecone import Pinecone, ServerlessSpec
# from langchain.document_loaders import PyMuPDFLoader, DirectoryLoader
# from langchain.text_splitter import RecursiveCharacterTextSplitter
# from langchain.prompts import PromptTemplate
# from langchain.llms import CTransformers
# %pip install python-dotenv

C:\Users\Pulina\AppData\Roaming\Python\Python310\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\Pulina\AppData\Local\Programs\Python\Python310\lib\site-packages\triton\knobs.py:212: UserWarning: Failed to find cuobjdump.exe
  warnings.warn(f"Failed to find {binary}")
c:\Users\Pulina\AppData\Local\Programs\Python\Python310\lib\site-packages\triton\knobs.py:212: UserWarning: Failed to find nvdisasm.exe
  warnings.warn(f"Failed to find {binary}")
c:\Users\Pulina\AppData\Local\Programs\Python\Python310\lib\site-packages\triton\knobs.py:212: UserWarning: Failed to find cuobjdump.exe
  warnings.warn(f"Failed to find {binary}")
c:\Users\Pulina\AppData\Local\Programs\Python\Python310\lib\site-packages\triton\knobs.py:212: UserWarning: Failed to find nvdisasm.exe
  warnings.warn(f"Failed to find {binary}")
W1202 20:3

In [3]:
from dotenv import load_dotenv
import os

load_dotenv()
PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")

In [4]:
# Extract data from pdf
def load_pdf(data):
    loader = DirectoryLoader(data, glob="*.pdf", loader_cls=PyMuPDFLoader)

    documents = loader.load()

    return documents

In [6]:
# extracted_data
extracted_data = load_pdf("../data/")

In [7]:
# Create text chunks
def text_split(extracted_data):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)
    text_chunks = text_splitter.split_documents(extracted_data)

    return text_chunks

In [8]:
text_chunks = text_split(extracted_data)
print(f"Number of text chunks: {len(text_chunks)}")

Number of text chunks: 5777


In [9]:
# download embeddings model
def download_hugging_face_embeddings():
    model_name = "sentence-transformers/all-MiniLM-L6-v2"
    embeddings = HuggingFaceEmbeddings(model_name=model_name)

    return embeddings

In [10]:
embeddings = download_hugging_face_embeddings()

In [11]:
embeddings

HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, query_encode_kwargs={}, multi_process=False, show_progress=False)

In [12]:
query_result = embeddings.embed_query("Hello PuLeeNa")
print("Length",len(query_result))

Length 384


In [ ]:
query_result

[-0.03990147262811661,
 0.0040896618738770485,
 0.004573088139295578,
 0.03819186985492706,
 -0.1727973073720932,
 -0.0367298349738121,
 0.08871994912624359,
 -0.03081340901553631,
 -0.02484363317489624,
 -0.053716957569122314,
 0.013927995227277279,
 -0.07987657934427261,
 0.02863658033311367,
 -0.050012119114398956,
 -0.03471437096595764,
 0.0394587405025959,
 0.08454418182373047,
 -0.06269487738609314,
 -0.099276103079319,
 0.02077256701886654,
 -0.0738036185503006,
 0.005315256770700216,
 -0.09213661402463913,
 0.04488122835755348,
 -0.06001381576061249,
 -0.03701809048652649,
 0.0065983054228127,
 0.006714714225381613,
 -0.046713124960660934,
 -0.053346242755651474,
 -0.008269516751170158,
 -0.02638341300189495,
 0.09019513428211212,
 0.01382873859256506,
 0.05642198026180267,
 0.09472527354955673,
 -0.028321826830506325,
 -0.004132629372179508,
 0.043210987001657486,
 0.008781567215919495,
 -0.06319089233875275,
 -0.06603561341762543,
 -0.08285818248987198,
 -0.020790910348296165

In [13]:
# Initializing Pinecone
pc = Pinecone(api_key=PINECONE_API_KEY)
index_name = "medical-chatbotn"

# Create index if it doesn't exist
if index_name not in pc.list_indexes().names():
    pc.create_index(
        name=index_name,
        dimension=384,
        metric='cosine',
        spec=ServerlessSpec(
            cloud='aws',
            region="us-east-1"
        )
    )

# Get the index object
index = pc.Index(index_name)

# Check if index is empty
stats = index.describe_index_stats()
record_count = stats.get('total_vector_count', 0)

if record_count == 0:
    # If index is empty, add embeddings
    docsearch = PineconeVectorStore.from_texts(
        [t.page_content for t in text_chunks], 
        embeddings, 
        index_name=index_name,
        pinecone_api_key=PINECONE_API_KEY
    )
else:
    # If index has records, just connect to it
    docsearch = PineconeVectorStore.from_existing_index(
        index_name=index_name,
        embedding=embeddings
    )

In [15]:
prompt_template = """
Use the following pieces of information to answer the user's question.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

Context: {context}
Question: {question}

Only return the helpful answer below and nothing else.
Helper Answer:
"""

In [16]:
PROMPT=PromptTemplate(template=prompt_template, input_variables=["context", "question"])
chain_type_kwargs = {"prompt": PROMPT}

In [18]:
llm = CTransformers(model="../model/llama-2-7b-chat.Q4_0.gguf", 
                    model_type="llama",
                    config={"max_new_tokens": 512, "temperature":0.8},
                    local_files_only=True)

In [ ]:
# index_name = "medical-chatbotn"
# docsearch = PineconeVectorStore.from_existing_index(
#     index_name=index_name,
#     embedding=embeddings
# )

In [19]:
qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=docsearch.as_retriever(search_kwargs={'k': 2}),
    return_source_documents=True,
    chain_type_kwargs=chain_type_kwargs
)

In [ ]:
# while True:
#     user_input = input(f"Input Prompt:")
#     result = qa({"query": user_input})
#     print("Response : ", result['result'])

C:\Users\Pulina\AppData\Local\Temp\ipykernel_39928\45570682.py:3: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use invoke instead.
  result = qa({"query": user_input})


Response :  The number of employees at the corporation is 425.
Response :  Acne is a common skin disease characterized by pimples on the face, chest, and back. It occurs when the pores of the skin become clogged with oil, dead skin cells, and bacteria.
Response :  Acne is a common skin disease characterized by pimples on the face, chest, and back. It occurs when the pores of the skin become clogged with oil, dead skin cells, and bacteria.
